## Advanced topics in Int. Trade. 
The goal of this script is to build the dataset for the project of the course Advanced Topics in International Trade

At first, we want to get data for the trade between countries. 

In [3]:
# import libraries
import pandas as pd
import os
import numpy as np

ImportError: C extension: pandas._config not built. If you want to import pandas from the source directory, you may need to run 'python setup.py build_ext --force' to build the C extensions first.

In [ ]:
# arguments
exports_filename_2022_2017 = "exports_TradeData_6_11_2023_17_22_19.csv"
imports_filename_2022_2017 = "imports_TradeData_6_11_2023_17_25_3.csv"

exports_filename_2016_2011 = "exports_TradeData_6_11_2023_17_40_30.csv"
imports_filename_2016_2011 = "imports_TradeData_6_11_2023_17_31_55.csv"

trade_filename = "data.dta"
gravity_filename = r"gravity_1995_2014.dta"

country_code_filename = "iso-country-codes.csv"

In [2]:
folder_path = os.getcwd()
path_to_data = os.path.join(folder_path,'data')
path_to_data

'/Users/clement/Desktop/ULB/Doctorat/ECARES M2/Advanced topics in Economics/Project/Gravity_trade_2/src/data'

In [3]:
os.getcwd()

'/Users/clement/Desktop/ULB/Doctorat/ECARES M2/Advanced topics in Economics/Project/Gravity_trade_2/src'

#### Load data

In [4]:
country_code = pd.read_csv(os.path.join(path_to_data, country_code_filename))
country_code.rename(columns = {'English short name lower case' : 'Country'}, inplace = True)
country_code = country_code[['Country', 'Alpha-3 code']]

NameError: name 'pd' is not defined

In [6]:
mapping_years = {'2012 [YR2012]': '2012',
'2013 [YR2013]': '2013',
'2014 [YR2014]': '2014',
'2015 [YR2015]': '2015',
'2016 [YR2016]': '2016',
'2017 [YR2017]': '2017',
'2018 [YR2018]': '2018',
'2019 [YR2019]': '2019',
'2020 [YR2020]': '2020',
'2021 [YR2021]': '2021',
'2022 [YR2022]': '2022'
}

##### 1.1 Trade data 

In [7]:
gravity = pd.read_stata(os.path.join(path_to_data, gravity_filename))
gravity.t = gravity.t.astype(int)

In [8]:
# I need to join the data from 2015 to 2022
distance = gravity[['j','i','dist']].drop_duplicates()
distance.rename(columns={'i':'ReporterISO','j':'PartnerISO'}, inplace=True)

In [9]:
exports_2022_2017_trade = pd.read_csv(os.path.join(path_to_data,exports_filename_2022_2017), sep=',', encoding='latin-1')
exports_2016_2015_trade = pd.read_csv(os.path.join(path_to_data,exports_filename_2016_2011), sep=',',encoding='latin-1')
exports_2016_2015_trade = exports_2016_2015_trade[(exports_2016_2015_trade['RefYear'] == 2015) | (exports_2016_2015_trade['RefYear'] == 2016)]
exports_2022_2017_trade = exports_2022_2017_trade.loc[:, ['RefYear','ReporterISO', 'PartnerISO', 'PrimaryValue']]
exports_2016_2015_trade = exports_2016_2015_trade.loc[:, ['RefYear','ReporterISO', 'PartnerISO', 'PrimaryValue']]
exports_2014_1995_trade = gravity.loc[:, ['t','i', 'j', 'v']]
exports_2014_1995_trade.rename(columns={'t': 'RefYear','i':'ReporterISO','j':'PartnerISO','v':'PrimaryValue'}, inplace=True)
exports_2022_1995 = pd.concat([exports_2014_1995_trade, exports_2016_2015_trade, exports_2022_2017_trade], ignore_index=True)

In [10]:
# For each pair of country, join the distance
exports_2022_1995 = pd.merge(exports_2022_1995, distance, on=['ReporterISO', 'PartnerISO'], how='left')

In [44]:
exports_2022_1995.groupby('RefYear').count()

,ReporterISO,PartnerISO,PrimaryValue,dist,gdp_exporter,gdp_importer,gdp_i,gdp_j,BRI_i,BRI_j,WTO_i,WTO_j,OECD_i,OECD_j,BRICS_i,BRICS_j,BRI_OECD
RefYear,,,,,,,,,,,,,,,,,
1995,18273,18273,18273,17931,0,0,17229,17255,18273,18273,18273,18273,18273,18273,18273,18273,18273
1996,19758,19758,19758,19383,0,0,18527,18582,19758,19758,19758,19758,19758,19758,19758,19758,19758
1997,20855,20855,20855,20465,0,0,19466,19565,20855,20855,20855,20855,20855,20855,20855,20855,20855
1998,21323,21323,21323,20934,0,0,19912,20011,21323,21323,21323,21323,21323,21323,21323,21323,21323
1999,21760,21760,21760,21362,0,0,20313,20361,21760,21760,21760,21760,21760,21760,21760,21760,21760
2000,23177,23177,23177,22699,0,0,21631,21633,23177,23177,23177,23177,23177,23177,23177,23177,23177
2001,23653,23653,23653,23135,0,0,22087,21952,23653,23653,23653,23653,23653,23653,23653,23653,23653
2002,24195,24195,24195,23626,0,0,22562,22521,24195,24195,24195,24195,24195,24195,24195,24195,24195
2003,24595,24595,24595,24034,0,0,23040,22958,24595,24595,24595,24595,24595,24595,24595,24595,24595


##### 1.3 Add GDP weights

In [11]:
# I need to join the data from 2015 to 2022
gdp_1995_2014 = gravity[['t','j','i','gdp_i','gdp_j']].drop_duplicates()
gdp_1995_2014.rename(columns={'t':'year','i':'ReporterISO','j':'PartnerISO'}, inplace=True)

In [12]:
gdp1_filename = r"b96863f7-3fb5-4cdf-aefb-b60d5c0e9ef5_Data.csv"
gdp2_filename = r"b96863f7-3fb5-4cdf-aefb-b60d5c0e9ef5_Series - Metadata.csv"


In [13]:
# Load gdp data
def prepare_gdp_file(path_to_data, gdp_filename): 
    """Refactofing file.

    Args:
        path_to_data (_type_): _description_
        gdp_filename (_type_): _description_
    """
    gdp1 = pd.read_csv(os.path.join(path_to_data, gdp_filename),
    dtype={'GDP (current US$)':int}
    )
    gdp1 = gdp1[gdp1["Series Name"] == "GDP (current US$)"]
    gdp1.rename(columns=mapping_years, inplace=True)
    gdp1 = gdp1.drop(columns=['Country Name','Series Name', 'Series Code'])
    gdp1 = gdp1.set_index('Country Code')
    gdp1 = gdp1.stack().reset_index()
    gdp1.rename(columns = {'level_1':'RefYear', 0:'gdp'}, inplace=True)
    gdp1['RefYear'] = gdp1['RefYear'].astype(int)
    gdp1 = gdp1[gdp1['gdp'] != '..']
    gdp1['gdp'] = gdp1['gdp'].astype(float) 
    
    return gdp1

In [14]:
gdp1 = prepare_gdp_file(path_to_data, gdp1_filename)
gdp2 = prepare_gdp_file(path_to_data, gdp2_filename)
gdp = pd.concat([gdp1, gdp2], ignore_index=True)

In [15]:
# For each exporter country, join the yearly GDP
exports_2022_1995 = pd.merge(exports_2022_1995, gdp, left_on=['ReporterISO', 'RefYear'], right_on=['Country Code', 'RefYear'], how='left')
exports_2022_1995.rename(columns={'gdp':'gdp_exporter'}, inplace=True)

In [16]:
# For each exporter country, join the yearly GDP
exports_2022_1995 = pd.merge(exports_2022_1995, gdp, left_on=['PartnerISO', 'RefYear'], right_on=['Country Code', 'RefYear'], how='left')
exports_2022_1995.rename(columns={'gdp':'gdp_importer'}, inplace=True)

In [17]:
exports_2022_1995.drop('Country Code_x', axis=1, inplace=True)
exports_2022_1995.drop('Country Code_y', axis=1, inplace=True)

In [18]:
exports_2022_1995 = pd.merge(exports_2022_1995, gdp_1995_2014, left_on=['ReporterISO','PartnerISO','RefYear'], right_on=['ReporterISO','PartnerISO','year'], how='left')

In [19]:
exports_2022_1995['gdp_importer'] = exports_2022_1995['gdp_importer'] + exports_2022_1995['gdp_j']
exports_2022_1995['gdp_exporter'] = exports_2022_1995['gdp_exporter'] + exports_2022_1995['gdp_i']

In [20]:
exports_2022_1995.drop('year', axis=1, inplace=True)

##### 2.2  Add BRI_membership


In [21]:
BRI_filename = r"BRI_Regions.xlsx"
bri_membership = pd.read_excel(os.path.join(path_to_data, BRI_filename))
bri_membership.drop('CN', axis = 1, inplace = True)
# add country code !
bri_membership = pd.merge(bri_membership, country_code, left_on=['EN'], right_on=['Country'], how='left')
bri_membership['bri_member'] = np.ones(len(bri_membership)).astype('int')
bri_membership

,EN,Country,Alpha-3 code,bri_member
0,China,China,CHN,1
1,Afghanistan,Afghanistan,AFG,1
2,Albania,Albania,ALB,1
3,Algeria,Algeria,DZA,1
4,Angola,Angola,AGO,1
...,...,...,...,...
145,Venezuela,Venezuela,VEN,1
146,Vietnam,Vietnam,VNM,1
147,Yemen,Yemen,YEM,1
148,Zambia,Zambia,ZMB,1


In [22]:
# exporter BRI membership
exports_2022_1995 = pd.merge(exports_2022_1995, bri_membership.drop(['EN','Country'], axis=1), left_on=['ReporterISO'], right_on=['Alpha-3 code'], how='left')
exports_2022_1995.rename(columns={'bri_member':'BRI_i'}, inplace=True)
exports_2022_1995.drop(['Alpha-3 code'], axis=1, inplace = True)

# partner BRI membership
exports_2022_1995 = pd.merge(exports_2022_1995, bri_membership.drop(['EN','Country'], axis=1), left_on=['PartnerISO'], right_on=['Alpha-3 code'], how='left')
exports_2022_1995.rename(columns={'bri_member':'BRI_j'}, inplace=True)
exports_2022_1995.drop(['Alpha-3 code'], axis=1, inplace = True)

In [23]:
exports_2022_1995['BRI_i'].fillna(0, inplace=True)
exports_2022_1995['BRI_j'].fillna(0, inplace=True)

##### Add WTO and OECD membership

In [24]:
wto_filename = r"mem-obs-list.xlsx"
wto_membership = pd.read_excel(os.path.join(path_to_data, wto_filename))
# add country code !
wto_membership = pd.merge(wto_membership, country_code, left_on=['Members'], right_on=['Country'], how='left')
wto_membership['wto_member'] = np.ones(len(wto_membership)).astype('int')
wto_membership.drop(['Members','Country'],axis = 1, inplace = True)
wto_membership['wto_membership_year'] = pd.to_datetime(wto_membership['Membership Date'], format='%d %B %Y').dt.year
wto_membership['wto_membership_year'].fillna(0, inplace=True)
wto_membership['wto_membership_year'] = wto_membership['wto_membership_year'].astype(int)
wto_membership.dropna(subset=['Membership Date'], inplace=True)
wto_membership.drop('Membership Date',axis = 1,inplace = True)
wto_membership.head()

,Alpha-3 code,wto_member,wto_membership_year
0,AFG,1,2016
1,ALB,1,2000
2,AGO,1,1996
3,ATG,1,1995
4,ARG,1,1995


In [25]:
exports_2022_1995 = pd.merge(exports_2022_1995, wto_membership, left_on=['ReporterISO'], right_on=['Alpha-3 code'], how='left')
# Compare the dates to determine if each row is after the membership date
exports_2022_1995['WTO_i'] = exports_2022_1995['RefYear'] >= exports_2022_1995['wto_membership_year']

In [26]:
exports_2022_1995.drop('wto_membership_year',axis = 1,inplace = True)

In [27]:
# Same with importer
exports_2022_1995 = pd.merge(exports_2022_1995, wto_membership, left_on=['PartnerISO'], right_on=['Alpha-3 code'], how='left')
# Compare the dates to determine if each row is after the membership date
exports_2022_1995['WTO_j'] = exports_2022_1995['RefYear'] >= exports_2022_1995['wto_membership_year']
exports_2022_1995.drop(['Alpha-3 code_x','wto_member_x','Alpha-3 code_y','wto_member_y','wto_membership_year'], axis = 1, inplace = True)

In [28]:
exports_2022_1995.columns

Index(['RefYear', 'ReporterISO', 'PartnerISO', 'PrimaryValue', 'dist',
       'gdp_exporter', 'gdp_importer', 'gdp_i', 'gdp_j', 'BRI_i', 'BRI_j',
       'WTO_i', 'WTO_j'],
      dtype='object')

In [29]:
exports_2022_1995['WTO_i'] = exports_2022_1995['WTO_i'].replace({True: 1, False: 0})
exports_2022_1995['WTO_j'] = exports_2022_1995['WTO_j'].replace({True: 1, False: 0})

##### Add OECD membership

In [30]:
country_code['COUNTRY'] = country_code['Country'].str.upper()

In [31]:
wto_filename = r"mem-obs-list.xlsx"
oecd_membership = pd.read_excel(os.path.join(path_to_data, wto_filename), sheet_name='oecd')
# add country code !
oecd_membership = pd.merge(oecd_membership, country_code, left_on=['country'], right_on=['COUNTRY'], how='left')
oecd_membership['oecd_member'] = np.ones(len(oecd_membership)).astype('int')
oecd_membership.drop(['country'],axis = 1, inplace = True)
oecd_membership['oecd_membership_year'] = pd.to_datetime(oecd_membership['date'], format='%d %B %Y').dt.year
oecd_membership['oecd_membership_year'].fillna(0, inplace=True)
oecd_membership['oecd_membership_year'] = oecd_membership['oecd_membership_year'].astype(int)
oecd_membership.dropna(subset=['date'], inplace=True)
oecd_membership.drop('date',axis = 1,inplace = True)
oecd_membership.head()

,Country,Alpha-3 code,COUNTRY,oecd_member,oecd_membership_year
0,Australia,AUS,AUSTRALIA,1,1971
1,Austria,AUT,AUSTRIA,1,1961
2,Belgium,BEL,BELGIUM,1,1961
3,Canada,CAN,CANADA,1,1961
4,Chile,CHL,CHILE,1,2010


In [32]:
exports_2022_1995 = pd.merge(exports_2022_1995, oecd_membership, left_on=['ReporterISO'], right_on=['Alpha-3 code'], how='left')
# Compare the dates to determine if each row is after the membership date
exports_2022_1995['OECD_i'] = exports_2022_1995['RefYear'] >= exports_2022_1995['oecd_membership_year']
exports_2022_1995.drop(['Country', 'Alpha-3 code', 'COUNTRY', 'oecd_member', 'oecd_membership_year'],axis = 1, inplace = True)

In [33]:
exports_2022_1995 = pd.merge(exports_2022_1995, oecd_membership, left_on=['PartnerISO'], right_on=['Alpha-3 code'], how='left')
# Compare the dates to determine if each row is after the membership date
exports_2022_1995['OECD_j'] = exports_2022_1995['RefYear'] >= exports_2022_1995['oecd_membership_year']

In [34]:
exports_2022_1995.drop(['Country', 'Alpha-3 code', 'COUNTRY', 'oecd_member', 'oecd_membership_year'],axis = 1, inplace = True)

##### Add BRICS

In [35]:
wto_filename = r"mem-obs-list.xlsx"
brics_membership = pd.read_excel(os.path.join(path_to_data, wto_filename), sheet_name='brics')
# add country code !
brics_membership = pd.merge(brics_membership, country_code, on=['Country'], how='left')
brics_membership.drop(['Country'],axis = 1, inplace = True)
brics_membership['brics_membership_year'] = pd.to_datetime(brics_membership['Date'], format='%Y').dt.year
brics_membership['brics_membership_year'].fillna(0, inplace=True)
brics_membership['brics_membership_year'] = brics_membership['brics_membership_year'].astype(int)
brics_membership.dropna(subset=['Date'], inplace=True)
brics_membership.drop('Date',axis = 1,inplace = True)
brics_membership.head()

,Alpha-3 code,COUNTRY,brics_membership_year
0,RUS,RUSSIA,2009
1,CHN,CHINA,2009
2,IND,INDIA,2009
3,MEX,MEXICO,2011
4,BRA,BRAZIL,2009


In [36]:
exports_2022_1995 = pd.merge(exports_2022_1995, brics_membership, left_on=['ReporterISO'], right_on=['Alpha-3 code'], how='left')
# Compare the dates to determine if each row is after the membership date
exports_2022_1995['BRICS_i'] = exports_2022_1995['RefYear'] >= exports_2022_1995['brics_membership_year']

In [37]:
exports_2022_1995.columns

Index(['RefYear', 'ReporterISO', 'PartnerISO', 'PrimaryValue', 'dist',
       'gdp_exporter', 'gdp_importer', 'gdp_i', 'gdp_j', 'BRI_i', 'BRI_j',
       'WTO_i', 'WTO_j', 'OECD_i', 'OECD_j', 'Alpha-3 code', 'COUNTRY',
       'brics_membership_year', 'BRICS_i'],
      dtype='object')

In [38]:
exports_2022_1995.drop([ 'Alpha-3 code', 'COUNTRY','brics_membership_year'], axis = 1, inplace=True)

In [39]:
exports_2022_1995 = pd.merge(exports_2022_1995, brics_membership, left_on=['PartnerISO'], right_on=['Alpha-3 code'], how='left')
# Compare the dates to determine if each row is after the membership date
exports_2022_1995['BRICS_j'] = exports_2022_1995['RefYear'] >= exports_2022_1995['brics_membership_year']

In [40]:
exports_2022_1995.drop(['Alpha-3 code', 'COUNTRY', 'brics_membership_year'], axis = 1, inplace=True)

In [41]:
exports_2022_1995['OECD_i'] = exports_2022_1995['OECD_i'].replace({True: 1, False: 0})
exports_2022_1995['OECD_j'] = exports_2022_1995['OECD_j'].replace({True: 1, False: 0})
exports_2022_1995['BRICS_i'] = exports_2022_1995['BRICS_i'].replace({True: 1, False: 0})
exports_2022_1995['BRICS_j'] = exports_2022_1995['BRICS_j'].replace({True: 1, False: 0})

##### Compute other dummmies

In [42]:
exports_2022_1995['BRI_OECD'] = (exports_2022_1995['BRI_i'] == 1) & (exports_2022_1995['BRI_j'] == 1) & (exports_2022_1995['OECD_i'] == 1) & (exports_2022_1995['OECD_j'] == 1)
exports_2022_1995['BRI_OECD'] = exports_2022_1995['BRI_OECD'].astype(int)
# SAME WITH WTO
exports_2022_1995['BRI_OECD'] = (exports_2022_1995['BRI_i'] == 1) & (exports_2022_1995['BRI_j'] == 1) & (exports_2022_1995['BRICS_i'] == 1) & (exports_2022_1995['BRICS_j'] == 1)
exports_2022_1995['BRI_OECD'] = exports_2022_1995['BRI_OECD'].astype(int)

In [43]:
from pandas_profiling import ProfileReport
# Generate the ProfileReport
profile = ProfileReport(exports_2022_1995, title="Pandas Profiling Report", explorative=True)

# Save the report as an HTML file
path_to_save = os.path.join(r"./reports", 'dataset' + ".html")
profile.to_file(path_to_save)


/Users/clement/Desktop/ULB/Doctorat/ECARES M2/Advanced topics in Economics/Project/Gravity_trade_2/gravity_trade/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/var/folders/dk/28gb2q9512d3w1nv9b59mmz40000gn/T/ipykernel_7919/768970623.py:1: DeprecationWarning: `import pandas_profiling` is going to be deprecated by April 1st. Please use `import ydata_profiling` instead.
  from pandas_profiling import ProfileReport
Export report to file: 100%|██████████| 1/1 [00:00<00:00, 161.79it/s]
